# VGGish - Speech Commands - Generate Spectrograms

This processes each audio file from the Speech Commands dataset and generates the spectrogram used by VGGish

In [1]:
import sys
sys.path.append("/home/ubuntu/odsc/vggish/lib/models/research/audioset/vggish")

In [2]:
import pandas as pd
import os

In [3]:
import vggish_params


In [4]:
vggish_params.EXAMPLE_HOP_SECONDS, vggish_params.EXAMPLE_WINDOW_SECONDS

(0.96, 0.96)

In [5]:
import numpy as np
import six
import soundfile
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Instructions for updating:
non-resource variables are not supported in the long term
Num GPUs Available:  0


In [6]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [7]:

import vggish_input
import vggish_postprocess
import vggish_slim

pca_params = '/home/ubuntu/odsc/vggish/lib/vggish_pca_params.npz'
ckpt = '/home/ubuntu/odsc/vggish/lib/vggish_model.ckpt'

In [6]:
wav_files = {
    'file_name' : [],
    'label': []
}

rootDir = '/home/ubuntu/audio/speech_commands'
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        if fname.endswith('.wav'):
            wav_files['label'].append(os.path.basename(dirName))
            wav_files['file_name'].append(os.path.join(dirName, fname))

df = pd.DataFrame(data=wav_files)

In [7]:
df.head()

file_name label
0  /home/ubuntu/audio/speech_commands/zero/8a90cf...  zero
1  /home/ubuntu/audio/speech_commands/zero/173ae7...  zero
2  /home/ubuntu/audio/speech_commands/zero/eb76bc...  zero
3  /home/ubuntu/audio/speech_commands/zero/978240...  zero
4  /home/ubuntu/audio/speech_commands/zero/246328...  zero

In [37]:
df.shape

(105835, 2)

In [8]:
df['label'].value_counts()

five                  4052
zero                  4052
yes                   4044
seven                 3998
no                    3941
nine                  3934
down                  3917
one                   3890
go                    3880
two                   3880
stop                  3872
six                   3860
on                    3845
left                  3801
eight                 3787
right                 3778
off                   3745
four                  3728
three                 3727
up                    3723
dog                   2128
wow                   2123
house                 2113
marvin                2100
bird                  2064
happy                 2054
cat                   2031
sheila                2022
bed                   2014
tree                  1759
backward              1664
visual                1592
follow                1579
learn                 1575
forward               1557
_background_noise_       6
Name: label, dtype: int64

In [9]:
fname = df['file_name'][0]
fname

'/home/ubuntu/audio/speech_commands/zero/8a90cf67_nohash_0.wav'

In [10]:
ex = vggish_input.wavfile_to_examples(fname)

In [11]:
ex.shape

(1, 96, 64)

In [12]:
audio_data = np.empty((df.shape[0], 96, 64))

In [44]:
for ind, row in df.iterrows():
    data = vggish_input.wavfile_to_examples(row.file_name)
    if data.shape[0] != 1:
        df.loc[ind, 'valid'] = False
        continue
    audio_data[ind, :, :] = data
    df.loc[ind, 'valid'] = True
    if ind % 1000==0:
        print(ind)

0
1000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
36000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
61000
63000
65000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
81000
82000
83000
84000
85000
87000
88000
89000
90000
91000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000


In [46]:
df.to_csv('wavfile_df.csv')

In [47]:
with open('wavfile_spec.dat', 'wb') as f:
    audio_data.tofile(f)